In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification


In [39]:
# Load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    sentences = [item['request'] for item in data]  # Extracting sentences from the 'request' key
    labels = [[item[key] for key in ['is_order', 'is_refund', 'is_status', 'is_date_change', 'is_dest_change', 'is_weather', 'is_allowed']] for item in data]
    return sentences, labels



In [5]:
# for item in datastore:
#     requests.append(item['request'])
#     parameters = [item['is_order'],item['is_refund'],item['is_status'],item['is_date_change'],item['is_dest_change'],item['is_weather'],item['is_allowed']]
#     full.append(parameters)
#     labels.append(parameters.index(1))
# print(labels)


[3, 5, 4, 3, 2, 4, 5, 4, 6, 6, 3, 2, 1, 0, 6, 2, 2, 4, 3, 5, 4, 2, 4, 6, 6, 3, 2, 1, 0, 6, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6]


In [40]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")


In [41]:
# Load data
sentences, labels = load_data("data/data.json")


In [42]:
# Tokenize and encode data
max_length = 64  # Adjust according to your data and BERT model's maximum input length
encoded_data = tokenizer(sentences, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

# Convert BatchEncoding to dictionary
encoded_data = {key: encoded_data[key].numpy() for key in encoded_data}

In [43]:
# Convert labels to integers (no one-hot encoding needed)
labels = np.array(labels)
print(labels)

[[1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 

In [44]:
#split the data into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded_data['input_ids'], labels, test_size=0.2, random_state=42)


In [46]:
# Load pre-trained BERT model
num_classes = 7 
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_classes)

SystemError: deallocated bytearray object has exported buffers

MemoryError: 

PanicException: Python API call failed

In [36]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [37]:
# Train the model
batch_size = 16
epochs = 3  # Adjust according to your training needs

history = model.fit(
    train_inputs,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1
)

Epoch 1/3


ValueError: in user code:

    File "c:\Users\nizan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\nizan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\nizan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\nizan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_tf_utils.py", line 1638, in train_step
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "c:\Users\nizan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\compile_utils.py", line 277, in __call__
  

In [13]:
# Make predictions
predictions = model.predict(encoded_data)
predicted_classes = tf.argmax(predictions.logits, axis=1).numpy()

# Convert predicted classes to original labels
# (assuming your original labels were 1-indexed)
predicted_labels = predicted_classes + 1 
print(predicted_labels)
# Print predictions
# print(f"original Label: {labels}")
# for sentence, predicted_label in zip(requests, predicted_labels):
#     print(f"Sentence: {sentence}")
#     print(f"Predicted Label: {predicted_label}")

#print model accuracy
accuracy = np.mean(labels == predicted_labels)
print(f"Accuracy: {accuracy:.2%}")

#test for one sentence
sentence = "האם מותר להעלות למטוס מזון?"
encoded_sentence = tokenizer(sentence, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
prediction = model.predict(encoded_sentence)
predicted_class = tf.argmax(prediction.logits, axis=1).numpy()
print(f"Sentence: {sentence}")
print(f"Predicted Label: {predicted_class+1}")

    

3/3 [==============================] - 4s 312ms/step
[4 3 3 3 3 3 3 1 3 3 1 3 3 3 3 3 3 1 3 3 3 3 1 3 3 1 3 3 1 3 3 1 3 1 1 1 3
 5 3 3 1 3 1 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3]
Accuracy: 13.24%
1/1 [==============================] - 3s 3s/step
Sentence: האם מותר להעלות למטוס מזון?
Predicted Label: [3]
